In [ ]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from elfpy.simulators import YieldSimulator
from elfpy.utils.data import format_trades

In [ ]:
pd.set_option("float_format",'{:,.6f}'.format)

In [ ]:
start_time = time.time()
num_runs = 0

config_file = "../../config/example_config.toml"
simulator = YieldSimulator(config_file)
simulator_rng = np.random.default_rng(simulator.config.simulator.random_seed)
simulator.reset_rng(simulator_rng)
simulator.set_random_variables()

apr = [0.10,]*config['num_trading_days']
override_dict = {
    'pricing_model_name': 'Element',
    'vault_apy': apr,
    'init_pool_apy' : apr[0],
    'init_vault_age': 1,
    'target_daily_volume': 5*1e5,
    'target_liquidity': 10*1e6,
    'fee_percent' : 0.1,
}
simulator.reset_rng(np.random.default_rng(simulator.config.simulator.random_seed)) # do this to make sure simulations run over the same trade sequence
simulator.run_simulation(override_dict)
num_runs += 1

floor_fee_list = [0, 5]
override_dict['pricing_model_name'] = 'HyperDrive'
for floor_fee in floor_fee_list:
    override_dict['floor_fee'] = floor_fee
    simulator.reset_rng(np.random.default_rng(simulator.config.simulator.random_seed)) # do this to make sure simulations run over the same trade sequence
    simulator.run_simulation(override_dict)
    num_runs += 1

end_time = time.time()
print(f'Total time for {num_runs} runs was {end_time-start_time:.3f} seconds; which is {(end_time-start_time)/num_runs:.3f} seconds per run')

In [ ]:
[trades, trades_agg] = format_trades(simulator.analysis_dict)

In [ ]:
# create runs dataframe from the last trade in each run (using time_diff instead would pick the 1st)
changing_columns = [ # These columns should change with run aggregation
    'run_number',
    'model_name',
    'num_orders',
    'pool_apy',
]
const_columns = [ # These should not change with run aggregation
    'init_time_stretch',
    'target_liquidity',
    'target_daily_volume',
    'fee_percent',
    'floor_fee',
    'init_vault_age',
    'vault_apy',
    'pool_duration',
    'num_trading_days'
] 
interesting_columns = changing_columns + const_columns
runs = trades.loc[trades.time_diff_shift > 0, :].loc[:, interesting_columns].copy()
# variables that change per trade represent the last value in the run, rename the useful ones
runs.rename(columns={'current_apy':'ending_apy', 'num_orders':'total_orders'}, inplace=True)
# there is one row per "run", set that as a named column
runs.set_index('run_number', inplace=True)
# display(runs.T)

In [ ]:
# target columns for taking averages & sums
mean_columns = [
    'time_until_end',
    'init_share_price',
    'base_asset_price',
    'spot_price',
    'out_without_fee_slippage',
    'base_asset_reserves',
    'out_without_fee',
    'fee',
    'trade_amount',
    'out_with_fee',
    'day',
    'fee_in_bps',
    'token_asset_reserves',
    'total_supply'
]
sum_columns = [
    'fee',
    'out_with_fee',
    'out_without_fee',
    'out_without_fee_slippage',
    'trade_amount'
]
trades_mean = trades.groupby(['run_number'])[mean_columns].mean()
trades_mean.columns = ['mean_'+col for col in trades_mean.columns]
trades_sum = trades.groupby(['run_number'])[sum_columns].sum()
trades_sum.columns = ['sum_'+col for col in trades_sum.columns]
# display(pd.concat([runs, trades_mean, trades_sum], axis=1).T)

In [ ]:
display(trades.groupby(['model_name', 'token_in']).agg({
    'trade_volume_usd': ['sum'],
    'run_trade_number': ['count'],
    'fee_in_bps': ['mean', 'std', 'min', 'max', 'sum'],
    'day': ['mean', 'min', 'max'],
    })
)

In [ ]:
numPlots = 7

figsize = (24, 18)
fig, ax = plt.subplots(ncols=1, nrows=numPlots, gridspec_kw = {'wspace':0, 'hspace':0, 'height_ratios':np.ones(numPlots)})
fig.patch.set_facecolor('white')   # set fig background color to white

colors = list(plt.rcParams['axes.prop_cycle'].by_key()['color'])
model_colors = {model:colors[i] for i, model in enumerate(trades_agg.model_name.unique())}

# fees
current_plot = 0
for model in trades_agg.model_name.unique():
    model_df = trades_agg.loc[trades_agg.model_name==model, :]
    ax[current_plot] = model_df.plot(
        x='day',
        y='fee_in_usd_sum',
        figsize=figsize,
        ax=ax[current_plot],
        color=model_colors[model],
        label=model
    )
ax[current_plot].set_xlabel('')
ax[current_plot].set_ylabel('Fees (USD)', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(
    visible=True,
    linestyle='--',
    linewidth='1',
    color='grey',
    which='both',
    axis='y'
)
ax[current_plot].xaxis.set_ticklabels([])
title = (
    f'Initial pool APY: {trades.pool_apy[0]:.2f}% Time Stretch: {trades.init_time_stretch[0]:.2f} Maturity: {trades.pool_duration[0]} days\n'
    +f'Target Liquidity: {trades.target_liquidity[0]:,.0f} Target Daily Volume: {trades.target_daily_volume[0]:,.0f} Percent Fees: {trades.fee_percent[0]:.2f}%'
)
ax[current_plot].set_title(title, fontsize=20)
ax[current_plot].legend(fontsize=18)

# fees cumulative
current_plot += 1
for model in trades_agg.model_name.unique():
    model_df = trades_agg.loc[trades_agg.model_name==model, :]
    ax[current_plot] = model_df.plot(
        x='day',
        y='fee_in_usd_cum_sum',
        figsize=figsize,
        ax=ax[current_plot],
        color=model_colors[model],
        label=f'{model}_cum {model_df.fee_in_usd_cum_sum.iloc[-1]:,.0f}')
ax[current_plot].set_xlabel('')
ax[current_plot].set_ylabel('Cumulative Fees (USD)', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(
    visible=True,
    linestyle='--',
    linewidth='1',
    color='grey',
    which='both',
    axis='y'
)
ax[current_plot].xaxis.set_ticklabels([])
ax[current_plot].legend(fontsize=18)

# pool APYs
current_plot += 1
for model in trades_agg.model_name.unique(): # for each model (per run?)
    model_df = trades.loc[trades.model_name==model, :]
    ax[current_plot] = model_df.plot(
        x='run_trade_number', # could also do num_orders if you want to know the apy for a given volume of trade
        y='pool_apy',
        figsize=figsize,
        ax=ax[current_plot],
        color=model_colors[model],
        label=f'{model}'
    )
ax[current_plot].set_xlabel('')
# ax[current_plot].set_ylabel('Pool APY', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(
    visible=True,
    linestyle='--',
    linewidth='1',
    color='grey',
    which='both',
    axis='y'
)
ax[current_plot].xaxis.set_ticklabels([])
ax[current_plot].legend(fontsize=18)

# vault APYs
#current_plot += 1
for model in [trades_agg.model_name.unique()[0]]: # only show the first model, vault APYs are the same for all models
    model_df = trades.loc[trades.model_name==model, :]
    ax[current_plot] = model_df.plot(
        x='run_trade_number',
        y='vault_apy',
        figsize=figsize,
        ax=ax[current_plot],
        linestyle='-',
        color='black',
        label=f'vault'
    )
ax[current_plot].set_xlabel('')
ax[current_plot].set_ylabel('APY', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y')
ax[current_plot].xaxis.set_ticklabels([])
ax[current_plot].legend(fontsize=18)

# trade volume
current_plot += 1
for model in trades_agg.model_name.unique():
    model_df = trades_agg.loc[trades_agg.model_name==model,:]
    ax[current_plot] = model_df.plot(
        x='day',
        y='trade_volume_usd_sum',
        kind='line',
        ax=ax[current_plot],
        color=model_colors[model],
        label=model
    )
ax[current_plot].set_xlabel('')
ax[current_plot].set_ylabel('Volume (USD)', fontsize=18)
ax[current_plot].tick_params(axis = 'both', labelsize=12)
ax[current_plot].grid(
    visible=True,
    linestyle='--',
    linewidth='1',
    color='grey',
    which='both',
    axis='y'
)
ax[current_plot].ticklabel_format(style='plain', axis='y')
ax[current_plot].legend(fontsize=18)

# reserves
current_plot += 1
for model in trades_agg.model_name.unique():
    ax[current_plot] = trades.loc[trades.model_name==model,:].plot(
        x='day',
        y='base_asset_reserves',
        kind='line',
        ax=ax[current_plot],
        color=model_colors[model],
        linestyle='-',
        label=model+'_x'
    )
    ax[current_plot] = trades.loc[trades.model_name==model,:].plot(
        x='day',
        y='token_asset_reserves',
        kind='line',
        ax=ax[current_plot],
        color=model_colors[model],
        linestyle='--',
        label=model+'_y'
    )
ax[current_plot-2].legend(fontsize=18)
ax[current_plot].set_xlabel('')
ax[current_plot].set_ylabel('Liquidity (USD)', fontsize=18)
ax[current_plot].tick_params(axis = 'both', labelsize=12)
ax[current_plot].grid(visible=True,linestyle='--', linewidth='1', color='grey', which='both', axis='y')
ax[current_plot].ticklabel_format(style='plain', axis='y')
ax[current_plot].legend(fontsize=18)

# price per share
current_plot += 1
for model in trades.model_name.unique():
    model_df = trades.loc[trades.model_name==model]
    ax[current_plot] = model_df.plot(
        x='day',
        y='share_price',
        figsize=figsize,
        ax=ax[current_plot],
        kind='line',
        color=model_colors[model],
        label=(
            f'{model} growth={model_df.loc[:, "share_price"].values[-1] / model_df.loc[:, "share_price"].values[0] - 1:.3%}')
    )
ax[current_plot].set_ylabel('Price per share', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(
    visible=True,
    linestyle='--',
    linewidth='1',
    color='grey',
    which='both',
    axis='y'
)
# ax[current_plot].xaxis.set_ticklabels([])
ax[current_plot].legend(fontsize=18)

# spot price
current_plot += 1
for model in trades.model_name.unique():
    model_df = trades.loc[trades.model_name==model]
    ax[current_plot] = model_df.plot(
        x='day',
        y='price_total_return_scaled_to_share_price',
        figsize=figsize,
        ax=ax[current_plot],
        kind='line',
        color=model_colors[model],
        label=(
            f'{model} growth={model_df.loc[:, "price_total_return_scaled_to_share_price"].values[-1] / model_df.loc[:, "price_total_return_scaled_to_share_price"].values[0]-1:.3%}')
    )
ax[current_plot].set_ylabel('Spot price', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(
    visible=True,
    linestyle='--',
    linewidth='1',
    color='grey',
    which='both',
    axis='y'
)
# ax[current_plot].xaxis.set_ticklabels([])
ax[current_plot].legend(fontsize=18)
ax[current_plot].set_xlabel('Day', fontsize=18)

plt.show()